In [2]:
import tensorflow as tf
import pickle
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle/House Prices/data/test.csv", index_col=0)

In [49]:
df = df[['OverallQual', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd','GarageType', 'GarageFinish', 'GarageCars',
       'GarageArea']]

In [33]:
df.isna().sum()

OverallQual      0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
GrLivArea        0
FullBath         0
TotRmsAbvGrd     0
GarageType      76
GarageFinish    78
GarageCars       1
GarageArea       1
dtype: int64

In [10]:
scaler = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Kaggle/House Prices/data/scaler.pkl", "rb"))
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Kaggle/House Prices/tf_House_Price_Model.h5")

In [50]:
qualMap = {
    "Ex": 4,
    "Gd": 3,
    "TA": 2,
    "Fa": 1,
    "Po": 0
}

df["GarageFinish"].fillna("Unf", inplace=True)

df[["GarageCars", "GarageArea"]] = df[["GarageCars", "GarageArea"]].fillna(0)

df["HeatingQC"] = df["HeatingQC"].map(qualMap).astype(np.int8)

df["CentralAir"] = df.CentralAir.map({"Y":True, "N":False}).astype(np.int8)

df["Electrical"] = df.Electrical.apply(lambda x: x == "SBrkr").astype(np.int8)

df["GarageFinish"] = df.GarageFinish.map({"Unf":0, "RFn": 1, "Fin":2})

df["GarageType"] = df.GarageType.fillna("None")

df = df.join(pd.get_dummies(df.GarageType))

df.drop("GarageType", inplace=True, axis=1)

df[['OverallQual', 'HeatingQC', '1stFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageFinish', 'GarageCars',
       'GarageArea']] = scaler.transform(df[['OverallQual', 'HeatingQC', '1stFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageFinish', 'GarageCars',
       'GarageArea']])

In [51]:
df.isna().sum()

OverallQual     0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
GrLivArea       0
FullBath        0
TotRmsAbvGrd    0
GarageFinish    0
GarageCars      0
GarageArea      0
2Types          0
Attchd          0
Basment         0
BuiltIn         0
CarPort         0
Detchd          0
None            0
dtype: int64

In [52]:
predictions = model.predict(df)

In [53]:
flat_predictions = [item for sublist in predictions for item in sublist]

In [56]:
output = pd.DataFrame({'Id': df.index, 'SalePrice': flat_predictions})

In [57]:
output

,Id,SalePrice
0,1461,125197.968750
1,1462,159879.140625
2,1463,155691.343750
3,1464,174899.953125
4,1465,221213.578125
...,...,...
1454,2915,101057.984375
1455,2916,110017.578125
1456,2917,146379.421875
1457,2918,112572.671875


In [58]:
output.to_csv('submission.csv', index=False)